In [ ]:
%cd /content/AML
!git clone https://github.com/m-depascale/AnomalySegmentation_AML

[Errno 2] No such file or directory: '/content/AML'
/content
Cloning into 'AnomalySegmentation_AML'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 336 (delta 8), reused 13 (delta 8), pack-reused 320
Receiving objects: 100% (336/336), 185.59 MiB | 29.52 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [ ]:
!pip install ood-metrics
!pip install visdom
!pip install numpy
!pip install matplotlib
!pip install Pillow
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=adba54b72120b0db9d27c87a5a98325d2ad19c94dcbfc586ae5ba7d534fdaa7a
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


# Load pretrained erfnet


In [ ]:
%cd /content/AnomalySegmentation_AML/train

/content/AnomalySegmentation_AML/train


In [ ]:
from erfnet import ERFNet
import torch
from torchsummary import summary

# Create an instance of BiSeNet model

def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model

erf = ERFNet(20)
model = load_my_state_dict(erf, torch.load('/content/AnomalySegmentation_AML/trained_models/erfnet_pretrained.pth')) #, map_location=lambda storage, loc: storage))

# Use GPU
device = torch.device("cuda") # "cpu" or "cuda"
model.to(device)


ERFNet(
  (encoder): Encoder(
    (initial_block): DownsamplerBlock(
      (conv): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layers): ModuleList(
      (0): DownsamplerBlock(
        (conv): Conv2d(16, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1-5): 5 x non_bottleneck_1d(
        (conv3x1_1): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (conv1x3_1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (conv3x1_2): Conv2d(64

In [ ]:
for name, module in model.named_modules():
    print(name)


encoder
encoder.initial_block
encoder.initial_block.conv
encoder.initial_block.pool
encoder.initial_block.bn
encoder.layers
encoder.layers.0
encoder.layers.0.conv
encoder.layers.0.pool
encoder.layers.0.bn
encoder.layers.1
encoder.layers.1.conv3x1_1
encoder.layers.1.conv1x3_1
encoder.layers.1.bn1
encoder.layers.1.conv3x1_2
encoder.layers.1.conv1x3_2
encoder.layers.1.bn2
encoder.layers.1.dropout
encoder.layers.2
encoder.layers.2.conv3x1_1
encoder.layers.2.conv1x3_1
encoder.layers.2.bn1
encoder.layers.2.conv3x1_2
encoder.layers.2.conv1x3_2
encoder.layers.2.bn2
encoder.layers.2.dropout
encoder.layers.3
encoder.layers.3.conv3x1_1
encoder.layers.3.conv1x3_1
encoder.layers.3.bn1
encoder.layers.3.conv3x1_2
encoder.layers.3.conv1x3_2
encoder.layers.3.bn2
encoder.layers.3.dropout
encoder.layers.4
encoder.layers.4.conv3x1_1
encoder.layers.4.conv1x3_1
encoder.layers.4.bn1
encoder.layers.4.conv3x1_2
encoder.layers.4.conv1x3_2
encoder.layers.4.bn2
encoder.layers.4.dropout
encoder.layers.5
encoder.l

# Training Loop

In [ ]:
%cd /content/AnomalySegmentation_AML/train

import os
import random
import time
import numpy as np
import torch
import math

from PIL import Image, ImageOps
from argparse import ArgumentParser

from torch.optim import SGD, Adam, lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad
from torchvision.transforms import ToTensor, ToPILImage

from dataset import VOC12,cityscapes
from transform import Relabel, ToLabel, Colorize
from visualize import Dashboard

import importlib
from iouEval import iouEval, getColorEntry

from shutil import copyfile

NUM_CHANNELS = 3
NUM_CLASSES = 20 #pascal=22, cityscapes=20

color_transform = Colorize(NUM_CLASSES)
image_transform = ToPILImage()

#Augmentations - different function implemented to perform random augments on both image and target
class MyCoTransform(object):
    def __init__(self, enc, augment=True, height=512):
        self.enc=enc
        self.augment = augment
        self.height = height
        pass
    def __call__(self, input, target):
        # do something to both images
        input =  Resize(self.height, Image.BILINEAR)(input)
        target = Resize(self.height, Image.NEAREST)(target)

        if(self.augment):
            # Random hflip
            hflip = random.random()
            if (hflip < 0.5):
                input = input.transpose(Image.FLIP_LEFT_RIGHT)
                target = target.transpose(Image.FLIP_LEFT_RIGHT)

            #Random translation 0-2 pixels (fill rest with padding
            transX = random.randint(-2, 2)
            transY = random.randint(-2, 2)

            input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
            target = ImageOps.expand(target, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
            target = target.crop((0, 0, target.size[0]-transX, target.size[1]-transY))

        input = ToTensor()(input)
        if (self.enc):
            target = Resize(int(self.height/8), Image.NEAREST)(target)
        target = ToLabel()(target)
        target = Relabel(255, 19)(target)

        return input, target


class CrossEntropyLoss2d(torch.nn.Module):

    def __init__(self, weight=None):
        super().__init__()

        self.loss = torch.nn.NLLLoss2d(weight)

    def forward(self, outputs, targets):
        return self.loss(torch.nn.functional.log_softmax(outputs, dim=1), targets)


def train(args, model, enc=False):
    best_acc = 0

    #TODO: calculate weights by processing dataset histogram (now its being set by hand from the torch values)
    #create a loder to run all images and calculate histogram of labels, then create weight array using class balancing

    weight = torch.ones(NUM_CLASSES)
    if (enc):
        weight[0] = 2.3653597831726
        weight[1] = 4.4237880706787
        weight[2] = 2.9691488742828
        weight[3] = 5.3442072868347
        weight[4] = 5.2983593940735
        weight[5] = 5.2275490760803
        weight[6] = 5.4394111633301
        weight[7] = 5.3659925460815
        weight[8] = 3.4170460700989
        weight[9] = 5.2414722442627
        weight[10] = 4.7376127243042
        weight[11] = 5.2286224365234
        weight[12] = 5.455126285553
        weight[13] = 4.3019247055054
        weight[14] = 5.4264230728149
        weight[15] = 5.4331531524658
        weight[16] = 5.433765411377
        weight[17] = 5.4631009101868
        weight[18] = 5.3947434425354
    else:
        weight[0] = 2.8149201869965
        weight[1] = 6.9850029945374
        weight[2] = 3.7890393733978
        weight[3] = 9.9428062438965
        weight[4] = 9.7702074050903
        weight[5] = 9.5110931396484
        weight[6] = 10.311357498169
        weight[7] = 10.026463508606
        weight[8] = 4.6323022842407
        weight[9] = 9.5608062744141
        weight[10] = 7.8698215484619
        weight[11] = 9.5168733596802
        weight[12] = 10.373730659485
        weight[13] = 6.6616044044495
        weight[14] = 10.260489463806
        weight[15] = 10.287888526917
        weight[16] = 10.289801597595
        weight[17] = 10.405355453491
        weight[18] = 10.138095855713

    weight[19] = 1.0

    assert os.path.exists(args['datadir']), "Error: datadir (dataset directory) could not be loaded"

    co_transform = MyCoTransform(enc, augment=True, height=args['height'])#1024)
    co_transform_val = MyCoTransform(enc, augment=False, height=args['height'])#1024)
    dataset_train = cityscapes(args['datadir'], co_transform, 'train')
    dataset_val = cityscapes(args['datadir'], co_transform_val, 'val')

    loader = DataLoader(dataset_train, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=True)
    loader_val = DataLoader(dataset_val, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=False)

    if args['cuda']:
        weight = weight.cuda()
    criterion = CrossEntropyLoss2d(weight)
    print(type(criterion))

    savedir = args['savedir']

    if (enc):
        automated_log_path = savedir + "/automated_log_encoder.txt"
        modeltxtpath = savedir + "/model_encoder.txt"
    else:
        automated_log_path = savedir + "/automated_log.txt"
        modeltxtpath = savedir + "/model.txt"

    if (not os.path.exists(automated_log_path)):    #dont add first line if it exists
        with open(automated_log_path, "a") as myfile:
            myfile.write("Epoch\t\tTrain-loss\t\tTest-loss\t\tTrain-IoU\t\tTest-IoU\t\tlearningRate")

    with open(modeltxtpath, "w") as myfile:
        myfile.write(str(model))

    optimizer = Adam(model.parameters(), 2e-4, (0.9, 0.999),  eps=1e-08, weight_decay=1e-4)

    start_epoch = 1
    if args['resume']:
        #Must load weights, optimizer, epoch and best value.
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'

        assert os.path.exists(filenameCheckpoint), "Error: resume option was used but checkpoint was not found in folder"
        checkpoint = torch.load(filenameCheckpoint)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        print("=> Loaded checkpoint at epoch {})".format(checkpoint['epoch']))

    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5) # set up scheduler     ## scheduler 1
    lambda1 = lambda epoch: pow((1-((epoch-1)/args['num_epochs'])),0.9)  ## scheduler 2
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)                             ## scheduler 2

    if args['visualize'] and args['steps_plot'] > 0:
        board = Dashboard(8097)

    for epoch in range(start_epoch, args['num_epochs']+1):
        print("----- TRAINING - EPOCH", epoch, "-----")
        print(weight)
        scheduler.step(epoch)    ## scheduler 2

        epoch_loss = []
        time_train = []

        doIouTrain = args['iouTrain']
        doIouVal =  args['iouVal']

        if (doIouTrain):
            iouEvalTrain = iouEval(NUM_CLASSES)

        usedLr = 0
        for param_group in optimizer.param_groups:
            print("LEARNING RATE: ", param_group['lr'])
            usedLr = float(param_group['lr'])

        model.train()
        for step, (images, labels) in enumerate(loader):

            start_time = time.time()

            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images)
            targets = Variable(labels)
            outputs = model(inputs)
            #print(outputs.shape)


            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.data.item())
            time_train.append(time.time() - start_time)

            if (doIouTrain):
                iouEvalTrain.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            #print(outputs.size())
            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data

                board.image(image, f'input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_train) / len(time_train) / args['batch_size']))


        average_epoch_loss_train = sum(epoch_loss) / len(epoch_loss)

        iouTrain = 0
        if (doIouTrain):
            iouTrain, iou_classes = iouEvalTrain.getIoU()
            iouStr = getColorEntry(iouTrain)+'{:0.2f}'.format(iouTrain*100) + '\033[0m'
            print ("EPOCH IoU on TRAIN set: ", iouStr, "%")

        #Validate on 500 val images after each epoch of training
        print("----- VALIDATING - EPOCH", epoch, "-----")
        model.eval()
        epoch_loss_val = []
        time_val = []

        if (doIouVal):
            iouEvalVal = iouEval(NUM_CLASSES)

        for step, (images, labels) in enumerate(loader_val):
            start_time = time.time()
            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
            targets = Variable(labels, volatile=True)
            outputs = model(inputs)

            loss = criterion(outputs, targets[:, 0])
            epoch_loss_val.append(loss.data.item())
            time_val.append(time.time() - start_time)


            #Add batch to calculate TP, FP and FN for iou estimation
            if (doIouVal):
                iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data
                board.image(image, f'VAL input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'VAL target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss_val) / len(epoch_loss_val)
                print(f'VAL loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_val) / len(time_val) / args['batch_size']))


        average_epoch_loss_val = sum(epoch_loss_val) / len(epoch_loss_val)

        iouVal = 0
        if (doIouVal):
            iouVal, iou_classes = iouEvalVal.getIoU()
            iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
            print ("EPOCH IoU on VAL set: ", iouStr, "%")


        if iouVal == 0:
            current_acc = -average_epoch_loss_val
        else:
            current_acc = iouVal
        is_best = current_acc > best_acc
        best_acc = max(current_acc, best_acc)
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
            filenameBest = savedir + '/model_best_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'
            filenameBest = savedir + '/model_best.pth.tar'
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': str(model),
            'state_dict': model.state_dict(),
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filenameCheckpoint, filenameBest)

        #SAVE MODEL AFTER EPOCH
        if (enc):
            filename = f'{savedir}/model_encoder-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_encoder_best.pth'
        else:
            filename = f'{savedir}/model-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_best.pth'
        if args['epochs_save'] > 0 and step > 0 and step % args['epochs_save'] == 0:
            torch.save(model.state_dict(), filename)
            print(f'save: {filename} (epoch: {epoch})')
        if (is_best):
            torch.save(model.state_dict(), filenamebest)
            print(f'save: {filenamebest} (epoch: {epoch})')
            if (not enc):
                with open(savedir + "/best.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))
            else:
                with open(savedir + "/best_encoder.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))

        #SAVE TO FILE A ROW WITH THE EPOCH RESULT (train loss, val loss, train IoU, val IoU)
        #Epoch		Train-loss		Test-loss	Train-IoU	Test-IoU		learningRate
        with open(automated_log_path, "a") as myfile:
            myfile.write("\n%d\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.8f" % (epoch, average_epoch_loss_train, average_epoch_loss_val, iouTrain, iouVal, usedLr ))

    return(model)   #return model (convenience for encoder-decoder training)

def save_checkpoint(state, is_best, filenameCheckpoint, filenameBest):
    torch.save(state, filenameCheckpoint)
    if is_best:
        print ("Saving model as best")
        torch.save(state, filenameBest)

/content/AnomalySegmentation_AML/train


# Call training

In [ ]:
import logging
import logging.handlers
import sys
import os


savedir = f'../save/Erfnet_finetuned_20_classes'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 0,
    'batch_size' : 4,
    'cuda' : True,
    'savedir' : savedir,
    'resume': False,
    'num_epochs': 5,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': False
}

for name, param in model.named_parameters():
   splits = name.split('.')
   if str(splits[0]) == 'encoder':
      param.requires_grad = False
   else:
      param.requires_grad = True

erfnet_finetuned_20_classes = train(args, model)

encoder.initial_block.conv.weight
encoder.initial_block.conv.bias
encoder.initial_block.bn.weight
encoder.initial_block.bn.bias
encoder.layers.0.conv.weight
encoder.layers.0.conv.bias
encoder.layers.0.bn.weight
encoder.layers.0.bn.bias
encoder.layers.1.conv3x1_1.weight
encoder.layers.1.conv3x1_1.bias
encoder.layers.1.conv1x3_1.weight
encoder.layers.1.conv1x3_1.bias
encoder.layers.1.bn1.weight
encoder.layers.1.bn1.bias
encoder.layers.1.conv3x1_2.weight
encoder.layers.1.conv3x1_2.bias
encoder.layers.1.conv1x3_2.weight
encoder.layers.1.conv1x3_2.bias
encoder.layers.1.bn2.weight
encoder.layers.1.bn2.bias
encoder.layers.2.conv3x1_1.weight
encoder.layers.2.conv3x1_1.bias
encoder.layers.2.conv1x3_1.weight
encoder.layers.2.conv1x3_1.bias
encoder.layers.2.bn1.weight
encoder.layers.2.bn1.bias
encoder.layers.2.conv3x1_2.weight
encoder.layers.2.conv3x1_2.bias
encoder.layers.2.conv1x3_2.weight
encoder.layers.2.conv1x3_2.bias
encoder.layers.2.bn2.weight
encoder.layers.2.bn2.bias
encoder.layers.3.con

"\nstate_dict = model.state_dict()\nfor key, value in state_dict.items():\n  if 'encoder' in key:\n    #value.autorequires_grad(False)\n    print((keu))\n"